In [18]:
# ================================
# Student Management System (SMS)
# ================================
# This program implements:
#  - File-based secure login (users.txt)
#  - CRUD (create, read, update, and delete) for student records (students.txt)
#  - Activity logging with timestamps (activity_log.txt)
#  - A simple menu-driven interface

In [19]:
# Import the required modules 
import os         # Allows the communication with the operating system to efficiently work with files, folders and system settings.
import getpass    # Used for username and password handling. Asks for passwords securely in login scripts or authentication.
import datetime   # Lets you create timestamps, calculate differences between dates, format date etc.

In [20]:
# Testing the imported modules to make sure they were installed and working properly.
"os" in globals(), "getpass" in globals(), "datetime" in globals()


(True, True, True)

In [21]:
# Store file paths
# ---------------------
# File paths
# ---------------------
USER_FILE = "users.txt"
STUDENT_FILE = "student.txt"
LOG_FILE = "activity_log.txt"
# These variables store the filenames where user data, student records, and logs will be saved.

In [22]:
# AUTHENTICATION SYSTEM
""" 
This secures access to the Student Management System. It loads user credentials from a file, allows new
users to be savedm, and checks login details against records. Successful logins are recorded in an activity log, 
while invalid attempts are denied. This ensures only authorized users can manage student data.
"""

' \nThis secures access to the Student Management System. It loads user credentials from a file, allows new\nusers to be savedm, and checks login details against records. Successful logins are recorded in an activity log, \nwhile invalid attempts are denied. This ensures only authorized users can manage student data.\n'

In [23]:
# Log and Load User's fucntions

def log_activity(username, action):
    """Append one line to the log file with a timestamp, the user, and what they did (action)."""

    # Open the log file in "append" mode so we keep old entries and add a new one
    with open(LOG_FILE, "a") as log:
        log.write(f"{datetime.datetime.now()} - {username} - {action}\n")

def load_users():
    """Load users from the file into dictionary"""
    users = {}
    if os.path.exists(USER_FILE):           # Only try to read if the file is there
        with open(USER_FILE, "r") as f:
            for line in f:                  # Iterate line-by-line
                if "," in line:             # Expect "username,password"
                    uname, pwd = line.strip().split(",", 1)    # Split only on the first comma
                    users[uname] = pwd
    return users

In [24]:
# Save User function

def save_user(username, password):
    """Add a new line "username,password" to save a new user to USER_FILE"""    

    # Open in append mode so we don't overwrite existing users
    with open(USER_FILE, "a") as f:
        f.write(f"{username},{password}\n")

In [25]:
# Authenticate User function

def authenticate():
    """
    Authenticate user login. Prompt the console user for username and password input.
    Return the username if the credentials match; otherwise return None.
    """  
    users = load_users()                   
    username = input("Enter Username: ").strip()
    password = getpass.getpass("Enter Password: ").strip()

    # Check dictionary: key exists and stored password matches input
    if username in users and users[username] == password:
        log_activity(username, "logged in")
        print("\n:✅ Login Successful!\n")
        return username
    else:
        print("\n✖️ Invalid credentials!\n")
        return None

In [26]:
# STUDENT DATA MANAGEMENT (CRUD)
"""
The student data management fucntions handle records stored in a file. They allows users to add new students,
view all records, search for specific students, update details, and delete entries. This ensures accurate and 
organized management of student information.
"""

'\nThe student data management fucntions handle records stored in a file. They allows users to add new students,\nview all records, search for specific students, update details, and delete entries. This ensures accurate and \norganized management of student information.\n'

In [27]:
# Add Student function

def add_student(username):
    """ Create a new student record by asking for roll, name, grade, then append to STUDENT_FILE. """
    roll = input("Enter Roll Number: ").strip()
    name = input("Enter Name: ").strip()
    grade =  input("Enter Grade: ").strip()

    with open(STUDENT_FILE, "a") as f:
        f.write(f"{roll},{name},{grade}\n")

    log_activity(username, f"Added student {roll}")
    print("✅ Student added successfully!")

In [28]:
# View Student function

def view_students():
    """ Print all student records in a readable format. """
    if not os.path.exists(STUDENT_FILE):
        print("No records found")
        return

    with open(STUDENT_FILE, "r") as f:
        print("\n--- Student Records ---")
        for line in f:                          # Go through each line
            roll, name, grade, = line.strip().split(",")                
            print(f"Roll: {roll} | Name: {name} | Grade: {grade}") 

In [29]:
# Search Student function

def search_student():
    roll_no = input("Enter Roll Number to search: ").strip()
    found = False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            for line in f:
                parts = line.strip().split(",")      # Expect: roll,name,grade
                if len(parts) != 3:                  # Skip any wrong input/empty lines
                    continue
                roll, name, grade = parts            # Unpack fields
                if roll == roll_no:                  # Compare with target roll number
                    print(f"✅ Found: Roll: {roll} | Name: {name} | Grade: {grade}")
                    found = True
                    break             # Stop searching after first match

    if not found:                    # If it never matched
        print("❌ Student not found.")

In [30]:
# Update Student function

def update_student(username):
    """  
    Update a student's name and grade by roll number.
    Implementation:
      - Read all existing lines
      - Rewrite file, replacing the matching line with updated fields.
    """  
    roll_no = input("Enter Roll Number to update: ").strip()
    students = []         # All the lines are collected here
    updated = False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            students = f.readlines()          # Read file into a list of lines

        # Re-open the file for writing and recreate it line-by-line
        with open(STUDENT_FILE, "w") as f:
            for line in students:
                parts = line.strip().split(",")
                if len(parts) != 3:
                    f.write(line)
                    continue
                roll, name, grade = [x.strip() for x in parts]
                if roll == roll_no:
                    # Show current values and ask for new ones
                    print(f"Curent Data → Name: {name}, Grade: {grade}")
                    new_name = input("Enter new Name: ").strip()
                    new_grade = input("Enter new Grade: ").strip()
                    f.write(f"{roll},{new_name},{new_grade}\n")
                    updated = True
                    log_activity(username, f"Updated student {roll}")
                else:
                    f.write(f"{roll},{name},{grade}\n")          # Keep non-matching records as they are

    # User feedback
    if updated:
        print("✅ Student record updated.")
    else:
        print("✖️ Student not found.")

In [31]:
# Delete Student function

def delete_student(username):
    """ 
      - Remove one student record by roll number.
      - Implementation:
      - Read all lines
      - Rewrite the file with all lines except the one that matches the roll.
    """
    roll_no = input("Enter Roll Number to delete: ").strip()
    students = []
    deleted = False

    if os.path.exists(STUDENT_FILE):
        with open(STUDENT_FILE, "r") as f:
            students = f.readlines()

        with open(STUDENT_FILE, "w") as f:
            for line in students:
                # Skip enter or invalid lines
                parts = line.strip().split(",")
                if len(parts) != 3:
                    f.write(line)       # Keep invalid lines unchanged
                    continue

                roll, name, grade = [x.strip() for x in parts]
                if roll == roll_no:
                    deleted = True
                    # Log deletion
                    log_activity(username, f"Deleted student {roll}")
                else:
                    f.write(f"{roll},{name},{grade}\n")   

    if deleted:
        print("✅ Student record deleted.")
    else:
        print("✖️ Student not found.")

In [32]:
# Report Generation

def generate_report():
    """ 
    Read all student records and print:
      - total number of students
      - how many per grade
    """
    if not os.path.exists(STUDENT_FILE):
        print("No records available.").strip()
        return
    
    total = 0
    grades = {}
    with open(STUDENT_FILE, "r") as f:
        for line in f:
            roll, name, grade = line.strip().split(",")
            total += 1
            grades[grade] = grades.get(grade, 0) + 1       # Count by grade

    print("\n📊 student Report")
    print(f"Total Students: {total}")
    for g, count in grades.items():
        print((f"Grade {g}: {count} student(s)"))

In [33]:
# Main Program
""" 
The function below allow users to remove events, it prompts for the event title to delete. It filters the events 
list to exclude any event with a matching title (case-sensitive) and updates the list. A conformation message is 
displayed regardless of whether the event was found or not. 
"""

' \nThe function below allow users to remove events, it prompts for the event title to delete. It filters the events \nlist to exclude any event with a matching title (case-sensitive) and updates the list. A conformation message is \ndisplayed regardless of whether the event was found or not. \n'

In [35]:
def main(): 
    print("===== Student Management System =====")
    """ 1) Prompt to create an Admin, if no user exists then exit so you can log in next run. 
        2) Otherwise, authenticate. 3) Show the menu loop and route to the appropriate function. 
    """

# Ensure admin user exists 
    if not os.path.exists(USER_FILE): 
        print("No users found, Create an Admin account.")
        uname = input("Set Admin Username: ").strip()
        pwd = getpass.getpass("Set Admin Password: ").strip() 
        save_user(uname, pwd) 
        print("✅ Admin created! Please restart the program.")

    username = None 
    while not username: 
        username = authenticate() 
        # Menu 
    while True: 
        print("\n--- Main menu ---") 
        print("1. Add Student") 
        print("2. view All Students") 
        print("3. Search Student") 
        print("4. Update Student") 
        print("5. Delete Student") 
        print("6. Generate Report") 
        print("7. Logout & Exit")

        choice = input("Enter choice: ").strip()
            
        if choice == "1": 
            add_student(username) 
        elif choice == "2": 
            view_students() 
        elif choice == "3": 
            search_student() 
        elif choice == "4": 
            update_student(username) 
        elif choice == "5": 
            delete_student(username) 
        elif choice == "6": 
            generate_report() 
        elif choice == "7": 
            log_activity(username, "Logged out") 
            print("👋 Goodbye!") 
            break 
        else: 
            print("✖️ Invalid choice! Please try again.") 
                
if __name__ == "__main__": 
    main()

===== Student Management System =====
No users found, Create an Admin account.


Set Admin Username:  Lyon Qing
Set Admin Password:  ········


✅ Admin created! Please restart the program.


Enter Username:  Lyon Qing
Enter Password:  ········



:✅ Login Successful!


--- Main menu ---
1. Add Student
2. view All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


...

✅ Student record deleted.

--- Main menu ---
1. Add Student
2. view All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  6



📊 student Report
Total Students: 5
Grade A: 2 student(s)
Grade D: 1 student(s)
Grade C: 1 student(s)
Grade B: 1 student(s)

--- Main menu ---
1. Add Student
2. view All Students
3. Search Student
4. Update Student
5. Delete Student
6. Generate Report
7. Logout & Exit


Enter choice:  7


👋 Goodbye!
